In [1]:
!pip install shapely


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
from shapely.geometry import Polygon, Point
import json
from typing import Dict, Any, Optional
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def link_settlements_to_farmland(farmland_geojson: Dict[str, Any], settlement_geojson: Dict[str, Any]) -> Dict[str, Any]:
    """
    筆ポリゴンGeoJSONに農地集落の情報を紐付ける関数

    Args:
        farmland_geojson (Dict[str, Any]): 筆ポリゴンのGeoJSONデータ
        settlement_geojson (Dict[str, Any]): 農地集落のGeoJSONデータ

    Returns:
        Dict[str, Any]: 農地集落情報が追加された筆ポリゴンのGeoJSONデータ
    """
    try:
        # 農地集落ポリゴンの生成
        settlement_polygons = []
        for feature in settlement_geojson['features']:
            try:
                geometry = feature['geometry']
                if geometry['type'] == 'Polygon':
                    # 単一のポリゴン
                    coords = geometry['coordinates']
                    if isinstance(coords[0][0], (int, float)):
                        # 単一の座標リスト
                        polygon = Polygon([coords])
                    else:
                        # 複数の座標リスト（外部リングと内部リング）
                        polygon = Polygon(coords[0], holes=coords[1:] if len(coords) > 1 else None)
                elif geometry['type'] == 'MultiPolygon':
                    # 複数のポリゴンを合成
                    polygons = []
                    for poly_coords in geometry['coordinates']:
                        if isinstance(poly_coords[0][0], (int, float)):
                            polygons.append(Polygon([poly_coords]))
                        else:
                            polygons.append(Polygon(poly_coords[0], holes=poly_coords[1:] if len(poly_coords) > 1 else None))
                    polygon = polygons[0]  # 最初のポリゴンを使用
                else:
                    logger.warning(f"未対応のジオメトリタイプ: {geometry['type']}")
                    continue

                settlement_polygons.append({
                    'polygon': polygon,
                    'properties': feature['properties']
                })
            except Exception as e:
                logger.warning(f"農地集落ポリゴンの生成エラー: {str(e)}")
                continue

        # 各筆ポリゴンに農地集落情報を追加
        for feature in farmland_geojson['features']:
            try:
                farm_coords = feature['geometry']['coordinates'][0]
                farm_polygon = Polygon(farm_coords)
                farm_centroid = farm_polygon.centroid

                # 最も近い農地集落を探す
                min_distance = float('inf')
                nearest_settlement = None

                for settlement in settlement_polygons:
                    # まず含まれているかチェック
                    if settlement['polygon'].contains(farm_centroid):
                        nearest_settlement = settlement['properties']
                        break
                    
                    # 含まれていない場合は距離を計算
                    distance = farm_centroid.distance(settlement['polygon'])
                    if distance < min_distance:
                        min_distance = distance
                        nearest_settlement = settlement['properties']

                if nearest_settlement:
                    # 農地集落情報を追加
                    feature['properties'].update({
                        'Settlement_id': nearest_settlement['RCOM'],
                        'Settlement_name': nearest_settlement['RCOM_NAME'],
                        'Settlement_pref': nearest_settlement['PREF_NAME'],
                        'Settlement_city': nearest_settlement['CITY_NAME']
                    })

            except Exception as e:
                logger.warning(f"筆ポリゴン処理エラー: {str(e)}")
                continue

        return farmland_geojson

    except Exception as e:
        logger.error(f"処理エラー: {str(e)}")
        raise

def process_geojson_files(farmland_path: str, settlement_path: str, output_path: str) -> None:
    """
    GeoJSONファイルを読み込み、処理して保存する関数

    Args:
        farmland_path (str): 筆ポリゴンGeoJSONファイルのパス
        settlement_path (str): 農地集落GeoJSONファイルのパス
        output_path (str): 出力ファイルパス
    """
    try:
        # GeoJSONファイルの読み込み
        with open(farmland_path, 'r', encoding='utf-8') as f:
            farmland_geojson = json.load(f)
        
        with open(settlement_path, 'r', encoding='utf-8') as f:
            settlement_geojson = json.load(f)

        # 農地集落の紐付け処理
        result_geojson = link_settlements_to_farmland(farmland_geojson, settlement_geojson)

        # 結果の保存
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(result_geojson, f, ensure_ascii=False, indent=2)

        logger.info(f"処理が完了しました。出力ファイル: {output_path}")

    except Exception as e:
        logger.error(f"ファイル処理エラー: {str(e)}")
        raise

input_farmland_path = "../src/models/area_optimization/data/processed/merged_geojson/filtered_merged_polygon_with_farm_pin.geojson"
input_settlement_path = "../src/models/area_optimization/data/processed/rcom/rcom_decoded.geojson"
output_path = "../src/models/area_optimization/data/processed/merged_geojson/merged_polygon_and_farm_pin_and_settlement.geojson"

process_geojson_files(input_farmland_path, input_settlement_path, output_path)

INFO:__main__:処理が完了しました。出力ファイル: ../src/models/area_optimization/data/processed/merged_geojson/merged_polygon_and_farm_pin_and_settlement.geojson
